In [6]:
train = True

# Libs

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls /content/drive/MyDrive/vineyards-1/test

images	labels


In [5]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [1]:
! pip install roboflow
! pip install clearml
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122
! pip install cython

Looking in indexes: https://download.pytorch.org/whl/cu122


In [2]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu122/torch2.0/index.html


In [20]:
import os
import torch
import cv2
import random
import numpy as np
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from roboflow import Roboflow
from detectron2 import model_zoo
from clearml import Task, Logger

In [8]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=J35TDHSFXOCRAW5QIRR5YF4EN0OA8R
%env CLEARML_API_SECRET_KEY=BKihDZnjDtPZzlBwNpsE6LY6r6tpJqZIbZuKiECKAu2pB78knuK6voRWtcWoizSq_J8

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=J35TDHSFXOCRAW5QIRR5YF4EN0OA8R
env: CLEARML_API_SECRET_KEY=BKihDZnjDtPZzlBwNpsE6LY6r6tpJqZIbZuKiECKAu2pB78knuK6voRWtcWoizSq_J8


# Loads

In [12]:
from roboflow import Roboflow
rf = Roboflow(api_key="aN3GiBEKIwWclb6IUSNG")
project = rf.workspace("dovgal-vladislav").project("vineyards-erbnx")
version = project.version(1)
dataset = version.download("coco")
# Регистрация датасета в формате COCO
register_coco_instances("vineyards_train", {}, "/content/vineyards-1/train/_annotations.coco.json", "/content/vineyards-1/train")
register_coco_instances("vineyards_val", {}, "/content/vineyards-1/valid/_annotations.coco.json", "/content/vineyards-1/valid")

loading Roboflow workspace...
loading Roboflow project...


# Pathes

In [18]:
dataset_yaml = '/content/vineyards-1/data.yaml'
train_dir = '/content/vineyards-1/train/images'
val_dir = '/content/vineyards-1/valid'
test_dir = '/content/vineyards-1/test'
save_dir = os.path.join('/content/drive/MyDrive/vineyards_task')
best_dir = '/content/drive/MyDrive/vineyards_task/vine_best.pt'

# Train

In [21]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("vineyards_train",)
cfg.DATASETS.TEST = ("vineyards_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 300
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[07/19 13:19:05 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:01, 167MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[07/19 13:19:07 d2.engine.train_loop]: Starting training from iteration 0


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)


[07/19 13:19:17 d2.utils.events]:  eta: 0:01:25  iter: 19  total_loss: 1.219  loss_cls: 0.4778  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.5408  loss_rpn_loc: 0.2072    time: 0.3284  last_time: 0.3087  data_time: 0.0271  last_data_time: 0.0051   lr: 0.00016068  max_mem: 1942M
[07/19 13:19:29 d2.utils.events]:  eta: 0:01:20  iter: 39  total_loss: 0.4447  loss_cls: 0.0185  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.1964  loss_rpn_loc: 0.1899    time: 0.3238  last_time: 0.3422  data_time: 0.0120  last_data_time: 0.0135   lr: 0.00032718  max_mem: 1942M
[07/19 13:19:35 d2.utils.events]:  eta: 0:01:14  iter: 59  total_loss: 0.367  loss_cls: 0.001985  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.1742  loss_rpn_loc: 0.185    time: 0.3214  last_time: 0.3115  data_time: 0.0064  last_data_time: 0.0064   lr: 0.00049367  max_mem: 1942M
[07/19 13:19:42 d2.utils.events]:  eta: 0:01:09  iter: 79  total_loss: 0.3635  loss_cls: 0.00134  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.1674  loss

# Test